# Redis database cleanup

This notebook walks you through connecting to every Redis instance used by the project and clearing their contents. It is intended for development or staging environments where you need to start from an empty database.

> ⚠️ **Warning:** the cleanup operation is irreversible. Make sure you have an up-to-date backup before continuing.


## Prerequisites

* A reachable Redis server (listening on `localhost:6379` by default).
* Access credentials (if the Redis server requires them).
* A Python environment that includes the `redis` library and the project package shipped with this repository.


In [ ]:
import redis
from redis_helpers.utils import clean_all_databases

print("Libraries imported successfully.")


In [ ]:
# Connection parameters (update as needed)
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = None  # Set the password if the server requires it
USE_SSL = False        # Set to True when the Redis server is exposed via TLS/SSL

# Database mapping used by the project
DB_MAPPING = {
    "DATA": 0,
    "CAN": 1,
    "R": 2,
    "NR": 3,
    "CAR": 4,
    "AR": 5,
    "GP": 6,
    "BP": 7,
    "PR": 8,
    "AP": 9,
    "LOGS": 10,
}

BASE_REDIS_CONFIG = {
    "host": REDIS_HOST,
    "port": REDIS_PORT,
    "decode_responses": True,
    "ssl": USE_SSL,
}

if REDIS_PASSWORD not in (None, ""):
    BASE_REDIS_CONFIG["password"] = REDIS_PASSWORD

print("Base configuration ready. Update the values above if needed.")


In [ ]:
def connect_to_databases(base_config, db_mapping):
    '''Create a Redis connection for each entry in the mapping.'''
    connections = {}
    for name, db_id in db_mapping.items():
        params = dict(base_config)
        params["db"] = db_id
        try:
            conn = redis.Redis(**params)
            conn.ping()
        except redis.RedisError as exc:
            raise RuntimeError(f"Connection to Redis DB {db_id} ({name}) failed: {exc}") from exc
        connections[name] = conn
        print(f"Connected to Redis DB {db_id} ({name})")
    print(f"\nTotal active connections: {len(connections)}")
    return connections


def describe_database_sizes(connections):
    '''Return the number of keys stored in each database.'''
    summary = {}

    for name, conn in connections.items():
        try:
            summary[name] = conn.dbsize()
        except redis.RedisError as exc:
            summary[name] = f"Error: {exc}"
    return summary


connections = connect_to_databases(BASE_REDIS_CONFIG, DB_MAPPING)

pre_cleanup = describe_database_sizes(connections)
print("\nKeys present before cleanup:")
for name, size in pre_cleanup.items():
    print(f" - {name} (DB {DB_MAPPING[name]}): {size}")


## Manual confirmation

To avoid accidental deletions, run the next cell and type `YES` when prompted. Any other input will cancel the operation.


In [ ]:
confirmation = input("Type 'YES' to proceed with the full database cleanup: ").strip().upper()
if confirmation != "YES":
    raise SystemExit("Operation cancelled by the user.")

clean_all_databases(connections, DB_MAPPING)


In [ ]:
post_cleanup = describe_database_sizes(connections)
print("\nKeys present after cleanup:")
for name, size in post_cleanup.items():
    print(f" - {name} (DB {DB_MAPPING[name]}): {size}")


If every database reports `0` keys, the cleanup completed successfully.
